In [5]:
import os
source_dir = '/mnt/sda/julie/datasets/gastrointestinal/labeled-images/'
class_names = []
class_maps = {}
class_number = {}

for i, class_dir in enumerate(os.listdir(source_dir)):
    if not os.path.isdir(os.path.join(source_dir, class_dir)):
        continue
    #print(class_dir)
    sub_dir = os.path.join(source_dir, class_dir)
    #print(sub_dir)
    for _sub_dir in os.listdir(sub_dir):
        sub_sub_dir = os.path.join(sub_dir, _sub_dir)
        for _sub_sub_dir in os.listdir(sub_sub_dir):
            class_images = os.listdir(os.path.join(sub_sub_dir, _sub_sub_dir)) 
            print(_sub_sub_dir, len(class_images))
            class_number[_sub_sub_dir] = len(class_images)


dyed-resection-margins 989
dyed-lifted-polyps 1002
ulcerative-colitis-grade-1-2 11
polyps 1028
ulcerative-colitis-grade-0-1 35
ulcerative-colitis-grade-3 133
hemorrhoids 6
ulcerative-colitis-grade-2 443
ulcerative-colitis-grade-2-3 28
ulcerative-colitis-grade-1 201
impacted-stool 131
bbps-0-1 646
bbps-2-3 1148
ileum 9
retroflex-rectum 391
cecum 1009
barretts 41
esophagitis-b-d 260
barretts-short-segment 53
esophagitis-a 403
z-line 932
pylorus 999
retroflex-stomach 764


In [6]:
sorted_class_number = sorted(class_number.items(), key=lambda x:x[1], reverse=True)
print(sorted_class_number)
sorted_class_number = sorted_class_number[:14]
print(sorted_class_number)

[('bbps-2-3', 1148), ('polyps', 1028), ('cecum', 1009), ('dyed-lifted-polyps', 1002), ('pylorus', 999), ('dyed-resection-margins', 989), ('z-line', 932), ('retroflex-stomach', 764), ('bbps-0-1', 646), ('ulcerative-colitis-grade-2', 443), ('esophagitis-a', 403), ('retroflex-rectum', 391), ('esophagitis-b-d', 260), ('ulcerative-colitis-grade-1', 201), ('ulcerative-colitis-grade-3', 133), ('impacted-stool', 131), ('barretts-short-segment', 53), ('barretts', 41), ('ulcerative-colitis-grade-0-1', 35), ('ulcerative-colitis-grade-2-3', 28), ('ulcerative-colitis-grade-1-2', 11), ('ileum', 9), ('hemorrhoids', 6)]
[('bbps-2-3', 1148), ('polyps', 1028), ('cecum', 1009), ('dyed-lifted-polyps', 1002), ('pylorus', 999), ('dyed-resection-margins', 989), ('z-line', 932), ('retroflex-stomach', 764), ('bbps-0-1', 646), ('ulcerative-colitis-grade-2', 443), ('esophagitis-a', 403), ('retroflex-rectum', 391), ('esophagitis-b-d', 260), ('ulcerative-colitis-grade-1', 201)]


In [7]:
for i, class_name in enumerate(sorted_class_number):
    class_maps[class_name[0]] = i

In [11]:
def generate_pareto_distribution(largest_class_size, num_classes, imbalance_ratio):
    smallest_class_size = largest_class_size / imbalance_ratio
    
    # Calculate the ratio factor for the geometric series
    factor = (largest_class_size / smallest_class_size)**(1 / (num_classes - 1))
    
    class_instances = [round(largest_class_size / (factor**i)) for i in range(num_classes)]
    
    # Ensure the largest class has the exact number of instances
    class_instances[0] = largest_class_size
    
    return class_instances

# Parameters
largest_class_size = 998
num_classes = 14
imbalance_ratios = [20]

# Generate distributions for different imbalance ratios
distributions = []
for ir in imbalance_ratios:
    distribution = generate_pareto_distribution(largest_class_size, num_classes, ir)
    distributions.extend(distribution)
    print(f"Imbalance Ratio {ir}: {distribution} - Total Instances: {sum(distribution)}")

Imbalance Ratio 20: [998, 793, 629, 500, 397, 315, 250, 199, 158, 125, 100, 79, 63, 50] - Total Instances: 4656


In [12]:
distributions

[998, 793, 629, 500, 397, 315, 250, 199, 158, 125, 100, 79, 63, 50]

In [13]:
import random
seed = 1
random.seed(seed)
train_samples = []
val_samples = []
test_samples = []


for i, class_dir in enumerate(os.listdir(source_dir)):
    if not os.path.isdir(os.path.join(source_dir, class_dir)):
        continue
    #print(class_dir)
    sub_dir = os.path.join(source_dir, class_dir)
    #print(sub_dir)
    for _sub_dir in os.listdir(sub_dir):
        sub_sub_dir = os.path.join(sub_dir, _sub_dir)
        for _sub_sub_dir in os.listdir(sub_sub_dir):
            class_images = os.listdir(os.path.join(sub_sub_dir, _sub_sub_dir)) 
            if _sub_sub_dir not in class_maps.keys():
                continue
            random.shuffle(class_images)
            train_s = class_images[:distributions[class_maps[_sub_sub_dir]]]
            val_s = class_images[distributions[class_maps[_sub_sub_dir]]:distributions[class_maps[_sub_sub_dir]]+50]
            test_s = class_images[distributions[class_maps[_sub_sub_dir]]+50:distributions[class_maps[_sub_sub_dir]]+150]
            train_samples.extend(['%s/%s/%s/%s' %(class_dir, _sub_dir, _sub_sub_dir, s) for s in train_s])
            val_samples.extend(['%s/%s/%s/%s' %(class_dir, _sub_dir, _sub_sub_dir, s) for s in val_s])
            test_samples.extend(['%s/%s/%s/%s' %(class_dir, _sub_dir, _sub_sub_dir, s) for s in test_s])


In [14]:
len(train_samples), len(val_samples), len(test_samples)

(4656, 700, 1400)

In [15]:
dic = {}
for i, class_dir in enumerate(os.listdir(source_dir)):
    if not os.path.isdir(os.path.join(source_dir, class_dir)):
        continue
    #print(class_dir)
    sub_dir = os.path.join(source_dir, class_dir)
    #print(sub_dir)
    for _sub_dir in os.listdir(sub_dir):
        sub_sub_dir = os.path.join(sub_dir, _sub_dir)
        for _sub_sub_dir in os.listdir(sub_sub_dir):
            class_images = os.listdir(os.path.join(sub_sub_dir, _sub_sub_dir)) 
            if _sub_sub_dir not in class_maps.keys():
                continue
            for s in class_images:
                dic['%s/%s/%s/%s' %(class_dir, _sub_dir, _sub_sub_dir, s)] = class_maps[_sub_sub_dir]

In [16]:
import numpy as np
np.save('../numpy/kvasir/train_20.npy' , train_samples)
np.save('../numpy/kvasir/val_20.npy', val_samples)
np.save('../numpy/kvasir/test_20.npy', test_samples)
np.save('../numpy/kvasir/dic.npy', dic)

In [31]:
source_dir = '/mnt/sda/julie/projects/OpenMedLongTailed/configs/kvasir/20/'
for configs in os.listdir(source_dir):
    os.system('mv '+source_dir+configs+' '+source_dir+configs.replace('isic', 'kvasir'))

In [8]:
import os
import numpy as np
for dataset in os.listdir('../numpy/'):
    sub_dir = '../numpy/%s/' %dataset
    for file_name in os.listdir(sub_dir):
        if 'test' in file_name:
            train_file = np.load(sub_dir+file_name)
            print(dataset, file_name)
            print(len(train_file.tolist()))

nct test_500.npy
4500
nct test_200.npy
4500
nct test_100.npy
4500
medmnist organamnist_test.npy
17778
medmnist tissuemnist_test.npy
47280
medmnist pathmnist_test.npy
7180
medmnist organsmnist_test.npy
8827
medmnist bloodmnist_test.npy
3421
medmnist organcmnist_test.npy
8216
medmnist dermamnist_test.npy
2005
kvasir test_20.npy
1400
isic test_500.npy
800
isic test_200.npy
800
isic test.npy
5229
isic test_100.npy
800


In [3]:
valid_accuracy = [0.4, 0.5, 0.3, 0]
valid_accuracy = [min(valid_accuracy) if x == 0 else x for x in valid_accuracy]

In [4]:
valid_accuracy

[0.4, 0.5, 0.3, 0]

In [6]:
0.0 == 0

True